Dataset to use:
https://www.kaggle.com/datasets/paultimothymooney/stock-market-data

A site to get thechnical indicators from:
https://www.investopedia.com/articles/active-trading/011815/top-technical-indicators-rookie-traders.asp?utm_source=chatgpt.com

Piano:
- Trovare indicatori da usare su una singola stock
- Trovare indicatori da usare su tutto l indice (più semplici come mean e std sull ultimo dato e una window di 5, 10, 20) per esempio:
    - Market index price
    - Market index trading volume
- Creare un nuovo dataset che usa questi indicatori

Libreria da usare: pandas‑ta
- contiene 130 indicatori già fatti (usarne una 50ina)


Chat per spunto:
https://chatgpt.com/share/681e1a2a-a944-8013-834e-2ef66c01417b

# Dataset Creation

In this notebook we download our data from kaggle and then augment it with classic technical analysis indicators.
We will use pandas-ta, an extension of pandas for Technical Analysis. 

Pandas-ta is a Numerical Time Series Feature Generator where the Time Series data is biased towards Financial Market data; typical data includes columns named :"Open", "High", "Low", "Close", "Volume".

Dataset used: https://www.kaggle.com/datasets/paultimothymooney/stock-market-data

In [1]:
import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("paultimothymooney/stock-market-data")

# print("Path to dataset files:", path)

## Basic Exploration

This is a quick overview about how the library works and the process of feature enrichment applied to one stock

In [15]:
import numpy as np
# re–add the old alias so pandas_ta can import it
if not hasattr(np, "NaN"):
    np.NaN = np.nan     # Needed by pandas_ta

# We need to downgrade pandas to 1.5.3 for pandas_ta to work correctly
import pandas as pd
import pandas_ta as ta

df = pd.DataFrame() # Empty DataFrame

# Load data
df = pd.read_csv("data/sp500/csv/A.csv", sep=",")
# OR if you have yfinance installed
# df = df.ta.ticker("aapl")



In [16]:
# Print DataFrame structure
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5804 entries, 0 to 5803
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            5804 non-null   object 
 1   Low             5804 non-null   float64
 2   Open            5804 non-null   float64
 3   Volume          5804 non-null   int64  
 4   High            5804 non-null   float64
 5   Close           5804 non-null   float64
 6   Adjusted Close  5804 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 317.5+ KB
None
         Date        Low       Open    Volume       High      Close  \
0  18-11-1999  28.612303  32.546494  62546380  35.765381  31.473534   
1  19-11-1999  28.478184  30.713518  15234146  30.758226  28.880545   
2  22-11-1999  28.657009  29.551144   6577870  31.473534  31.473534   
3  23-11-1999  28.612303  30.400572   5975611  31.205294  28.612303   
4  24-11-1999  28.612303  28.701717   4843231  29.998213  29.3723

In [17]:
# VWAP requires the DataFrame index to be a DatetimeIndex.
# Replace "datetime" with the appropriate column from your DataFrame
df.set_index(pd.DatetimeIndex(df["Date"]), inplace=True)

# Drop the "Date" column if it's no longer needed
df.drop(columns=["Date"], inplace=True)

# Calculate Returns and append to the df DataFrame
# This automatically calculates the log return and percent return 
#  from the 'close' field in our dataset using the pandas-ta indicator
df.ta.log_return(cumulative=True, append=True)
df.ta.percent_return(cumulative=True, append=True)

# New Columns with results
df.columns

# Take a peek
df.tail()

# vv Continue Post Processing vv

C:\Users\Gabriele\AppData\Local\Temp\ipykernel_20328\2585312532.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df.set_index(pd.DatetimeIndex(df["Date"]), inplace=True)


,Low,Open,Volume,High,Close,Adjusted Close,CUMLOGRET_1,CUMPCTRET_1
Date,,,,,,,,
2022-06-12,149.910004,152.149994,1172300,153.000000,151.350006,151.350006,1.570448,3.808802
2022-07-12,151.130005,151.229996,1011100,154.350006,153.729996,153.729996,1.586051,3.884421
2022-08-12,153.479996,154.419998,1289900,156.990005,156.279999,156.279999,1.602502,3.965442
2022-09-12,152.679993,155.669998,1020200,156.699997,152.949997,152.949997,1.580964,3.859639
2022-12-12,153.380005,154.009995,188024,155.399994,155.399994,155.399994,1.596855,3.937482


In [18]:
# Help about this, 'ta', extension
# help(df.ta)

# List of all indicators
df.ta.indicators()

# # Help about an indicator such as bbands
# help(ta.bbands)

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

Pandas-ta offers a variety of technical indicators. To enrich our dataset, we will use the majority of them as additional features.

Pandas-ta offer pre-packed 'strategies' including a subset of indicators.

Two popular one are
- ta.CommonStrategy  (subset of the most commonly used)
- ta.AllStrategy     (this includes all the indicators, usually used for feature generation)

We will focus on ta.AllStrategy

In [19]:
# print dataset size
print("Dataset size:", df.shape)

Dataset size: (5804, 8)


In [20]:
# Common Strategy
print("Common Strategy")
print(ta.CommonStrategy)

for indicator in ta.CommonStrategy.ta:
    print(indicator)


# All Strategy
print("All Strategy")
print(ta.AllStrategy)
# While common strategy is a defined subset of all indicators, 
#  all strategy dinamically generates a list of all indicators from the current list
#  It is a special command



Common Strategy
Strategy(name='Common Price and Volume SMAs', ta=[{'kind': 'sma', 'length': 10}, {'kind': 'sma', 'length': 20}, {'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}, {'kind': 'sma', 'close': 'volume', 'length': 20, 'prefix': 'VOL'}], description='Common Price SMAs: 10, 20, 50, 200 and Volume SMA: 20.', created='Tuesday May 13, 2025, NYSE: 7:23:52, Local: 11:23:52 ora legale Europa occidentale, Day 133/365 (36.00%)')
{'kind': 'sma', 'length': 10}
{'kind': 'sma', 'length': 20}
{'kind': 'sma', 'length': 50}
{'kind': 'sma', 'length': 200}
{'kind': 'sma', 'close': 'volume', 'length': 20, 'prefix': 'VOL'}
All Strategy
Strategy(name='All', ta=None, description='All the indicators with their default settings. Pandas TA default.', created='Tuesday May 13, 2025, NYSE: 7:23:52, Local: 11:23:52 ora legale Europa occidentale, Day 133/365 (36.00%)')


Here we choose the indicators

In [21]:
# Select first 1000 elements in the dataframe
df = df.iloc[:1000]

# for indicator in ta.AllStrategy.ta:
df.ta.strategy(
    ta.AllStrategy,
    calculate=True,
    verbose=False,
)

131it [00:01, 66.42it/s]


In [9]:
# Print the first 5 rows to check the new columns
print(df.head())

                  Low       Open   Volume       High      Close  \
Date                                                              
2005-09-27  19.100000  21.049999   961200  21.400000  19.299999   
2005-09-28  19.200001  19.299999  5747900  20.530001  20.500000   
2005-09-29  20.100000  20.400000  1078200  20.580000  20.209999   
2005-09-30  20.180000  20.260000  3123300  21.049999  21.010000   
2005-03-10  20.900000  20.900000  1057900  21.750000  21.500000   

            Adjusted Close  CUMLOGRET_1  CUMPCTRET_1  ABER_ZG_5_15  \
Date                                                                 
2005-09-27       18.194910     0.000000     0.000000           NaN   
2005-09-28       19.326204     0.060320     0.062176           NaN   
2005-09-29       19.052805     0.046072     0.047150           NaN   
2005-09-30       19.807001     0.084893     0.088601           NaN   
2005-03-10       20.268940     0.107948     0.113990     20.487333   

            ABER_SG_5_15  ...  VIDYA_14

In [22]:
print(list(df.columns))

['Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close', 'CUMLOGRET_1', 'CUMPCTRET_1', 'ABER_ZG_5_15', 'ABER_SG_5_15', 'ABER_XG_5_15', 'ABER_ATR_5_15', 'ACCBL_20', 'ACCBM_20', 'ACCBU_20', 'AD', 'ADOSC_3_10', 'ADX_14', 'DMP_14', 'DMN_14', 'ALMA_10_6.0_0.85', 'AMATe_LR_8_21_2', 'AMATe_SR_8_21_2', 'AO_5_34', 'OBV', 'OBV_min_2', 'OBV_max_2', 'OBVe_4', 'OBVe_12', 'AOBV_LR_2', 'AOBV_SR_2', 'APO_12_26', 'AROOND_14', 'AROONU_14', 'AROONOSC_14', 'ATRr_14', 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0', 'BBB_5_2.0', 'BBP_5_2.0', 'BIAS_SMA_26', 'BOP', 'AR_26', 'BR_26', 'CCI_14_0.015', 'CDL_DOJI_10_0.1', 'CDL_INSIDE', 'open_Z_30_1', 'high_Z_30_1', 'low_Z_30_1', 'close_Z_30_1', 'CFO_9', 'CG_10', 'CHOP_14_1_100', 'CKSPl_10_3_20', 'CKSPs_10_3_20', 'CMF_20', 'CMO_14', 'COPC_11_14_10', 'CTI_12', 'LDECAY_5', 'DEC_1', 'DEMA_10', 'DCL_20_20', 'DCM_20_20', 'DCU_20_20', 'DPO_20', 'EBSW_40_10', 'EFI_13', 'EMA_10', 'ENTP_10', 'EOM_14_100000000', 'ER_10', 'BULLP_13', 'BEARP_13', 'FISHERT_9_1', 'FISHERTs_9_1', 'FWM

In [23]:
# Print all column names and their values for the 30th row using a for loop
row = df.iloc[200]
for col, val in row.items():
    print(f"{col}: {val}")

Low: 43.81258773803711
Open: 44.12553787231445
Volume: 2044994.0
High: 44.70672225952149
Close: 44.21495056152344
Adjusted Close: 37.83172225952149
CUMLOGRET_1: 0.3399159894355293
CUMPCTRET_1: 0.4048295650812219
ABER_ZG_5_15: 44.10169296264648
ABER_SG_5_15: 46.33022283947069
ABER_XG_5_15: 41.87316308582227
ABER_ATR_5_15: 2.2285298768242123
ACCBL_20: 33.68751549934111
ACCBM_20: 37.824123764038085
ACCBU_20: 41.656488659131156
AD: -33820652.1499736
ADOSC_3_10: 3624425.78438285
ADX_14: 30.970826395501806
DMP_14: 0.7698087535902579
DMN_14: 0.4178428459244688
ALMA_10_6.0_0.85: 42.84267125024127
AMATe_LR_8_21_2: 1.0
AMATe_SR_8_21_2: 0.0
AO_5_34: 8.156084167256076
OBV: 21058212.0
OBV_min_2: 21058212.0
OBV_max_2: 23103206.0
OBVe_4: 21750153.557137266
OBVe_12: 17452583.708339468
AOBV_LR_2: 1.0
AOBV_SR_2: 0.0
APO_12_26: 7.387212924468216
AROOND_14: 7.14285714285714
AROONU_14: 71.42857142857143
AROONOSC_14: 64.28571428571429
ATRr_14: 2.1868621148306424
BBL_5_2.0: 43.2519390599802
BBM_5_2.0: 44.214

We select only those indicator that never get a nan value after 35 features

In [24]:
nan_features = df.iloc[35:1000].isna().any()
features_with_nan = nan_features[nan_features].index.tolist()
print("Features with at least one NaN between rows 100 and 1000:")
for feature in features_with_nan:
    print(feature)

Features with at least one NaN between rows 100 and 1000:
DPO_20
EBSW_40_10
HILOl_13_21
HILOs_13_21
ISA_9
ISB_26
ICS_26
KST_10_15_20_30_10_10_10_15
KSTs_9
KVO_34_55_13
KVOs_34_55_13
PSARl_0.02_0.2
PSARs_0.02_0.2
QQEl_14_5_4.236
QQEs_14_5_4.236
SUPERTl_7_3.0
SUPERTs_7_3.0
TRIXs_30_9


In [25]:
# We drop the previous indicators
cols_to_drop = [
    "DPO_20",
    "EBSW_40_10",
    "HILOl_13_21",
    "HILOs_13_21",
    "ISA_9",
    "ISB_26",
    "ICS_26",
    "KST_10_15_20_30_10_10_10_15",
    "KSTs_9",
    "KVO_34_55_13",
    "KVOs_34_55_13",
    "PSARl_0.02_0.2",
    "PSARs_0.02_0.2",
    "QQEl_14_5_4.236",
    "QQEs_14_5_4.236",
    "SUPERTl_7_3.0",
    "SUPERTs_7_3.0",
    "TRIXs_30_9"
]
df = df.drop(columns=cols_to_drop, errors="ignore")


# We set remaining nan datapoints to 0
df = df.fillna(0)

# Drop the first 50 datapoints from the DataFrame
df = df.iloc[50:]

# Dataset Creation

In this section we will create a dataset of stocks active in between 2008 and 2022, enrich the dataset with new features and then create a second dataset with market informations for the same period.

In [ ]:
import numpy as np
# re–add the old alias so pandas_ta can import it
if not hasattr(np, "NaN"):
    np.NaN = np.nan     # Needed by pandas_ta

# We need to downgrade pandas to 1.5.3 for pandas_ta to work correctly
import pandas as pd
import pandas_ta as ta




Utility code to delete all files (i.e stocks) that do not have datapoints in between 01-10-2007 and 2022.

We will then compute all the metrics for 01-10-2007 onward and keep all the datapoints from 2008.
We start computing metrics from 01-10-2007, becouse some indicators lag in time.

In [59]:
# This just counts them for a specific folder
import os
import pandas as pd

csv_folder = os.path.join("data", "cleaned", "sp500", "csv")
files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

active_both_dates = []
not_active_both_dates = []

for f in files:
    path = os.path.join(csv_folder, f)
    try:
        df_tmp = pd.read_csv(path)
        # Check if both dates are present in the 'Date' column
        if "Date" in df_tmp.columns:
            dates = df_tmp["Date"].astype(str)
        else:
            # If no header, assume first column is date
            dates = df_tmp.iloc[:, 0].astype(str)
        if ("01-10-2007" in dates.values) and ("12-12-2022" in dates.values):
            active_both_dates.append(f)
        else:
            not_active_both_dates.append(f)
    except Exception as e:
        print(f"Could not read {f}: {e}")

print(f"Files NOT active in both 01-10-2007 and 12-12-2022: {len(not_active_both_dates)}")
for fname in not_active_both_dates:
    print(fname)

Files NOT active in both 01-10-2007 and 12-12-2022: 2
MSCI.csv
ULTA.csv


In [60]:
# This goes through all the folders and deletes the files that are not active in both dates

folders = ['forbes2000', 'nasdaq', 'nyse', 'sp500']

for folder in folders:
    csv_folder = os.path.join("data", "cleaned", folder, "csv")
    files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

    not_active_both_dates = []

    for f in files:
        path = os.path.join(csv_folder, f)
        try:
            df_tmp = pd.read_csv(path)
            if "Date" in df_tmp.columns:
                dates = df_tmp["Date"].astype(str)
            else:
                dates = df_tmp.iloc[:, 0].astype(str)
            if not (("01-10-2007" in dates.values) and ("12-12-2022" in dates.values)):
                not_active_both_dates.append(f)
        except Exception as e:
            print(f"Could not read {f}: {e}")
            # Delete unreadable file
            try:
                os.remove(path)
            except Exception as e2:
                print(f"Could not delete {f}: {e2}")

    print(f"{folder}: {len(not_active_both_dates)} files NOT active in both 01-10-2007 and 12-12-2022")

    # Delete the files not active in both dates
    for f in not_active_both_dates:
        try:
            os.remove(os.path.join(csv_folder, f))
        except Exception as e:
            print(f"Could not delete {f}: {e}")

forbes2000: 3 files NOT active in both 01-10-2007 and 12-12-2022
nasdaq: 8 files NOT active in both 01-10-2007 and 12-12-2022
nyse: 8 files NOT active in both 01-10-2007 and 12-12-2022
sp500: 2 files NOT active in both 01-10-2007 and 12-12-2022


In [ ]:
# Here we create the new dataset and stor it in the 'enriched' folder
# We use the same date format as the original dataset
# We keep data from 02-01-2008 to 12-12-2022

input_folder = os.path.join("data", "cleaned", "sp500", "csv")
output_folder = os.path.join("data", "enriched", "sp500", "csv")
os.makedirs(output_folder, exist_ok=True)

files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

for f in files:
    input_path = os.path.join(input_folder, f)
    output_path = os.path.join(output_folder, f)
    try:
        df = pd.read_csv(input_path, sep=",")
        stock_symbol = os.path.splitext(f)[0]

        # Ensure Date is in datetime format and set as index, then drop the column
        if "Date" in df.columns:
            df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y", errors="coerce")
            date_idx = df.columns.get_loc("Date")
            df.insert(date_idx + 1, "Stock_symbol", stock_symbol)
            df.set_index(pd.DatetimeIndex(df["Date"]), inplace=True)
            df.drop(columns=["Date"], inplace=True)
        else:
            df.insert(1, "Stock_symbol", stock_symbol)

        # Apply AllStrategy
        df.ta.strategy(
            ta.AllStrategy,
            calculate=True,
            verbose=False,
        )

        # Filter rows from 02-01-2008 onwards (index is now DatetimeIndex)
        
        df = df[df.index >= pd.to_datetime("02-01-2008", format="%d-%m-%Y")].copy()

        # Set all NaN values to 0
        df = df.fillna(0)

        # Save to new folder
        df.to_csv(output_path)
        print(f"Enriched and saved {f}")
    except Exception as e:
        print(f"Could not process {f}: {e}")

### Market Information

Here we compute the mean return of the day (ret_1), the rolling means of the mean returns over 5, 10 and 30 days and the rolling mean/std of the mean volume over 5, 10 and 30 days.

In [70]:
def compute_index_features(input_folder, prefix):
    files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]
    dfs = []
    for f in files:
        path = os.path.join(input_folder, f)
        try:
            df = pd.read_csv(path, sep=",")
            if "Date" in df.columns:
                df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y", errors="coerce")
                df = df.set_index("Date")
            dfs.append(df)
        except Exception as e:
            print(f"Could not read {f}: {e}")
    all_stocks = pd.concat(dfs)
    all_stocks = all_stocks[all_stocks.index.notnull()]
    agg = all_stocks.groupby(all_stocks.index).agg({
        "Close": ["mean", "std"],
        "Volume": ["mean", "std"],
    })
    agg.columns = ['_'.join(col) for col in agg.columns]
    agg = agg.sort_index()
    agg[f"ret_1_{prefix}"] = agg[f"Close_mean"].pct_change()
    for w in [5, 10, 30]:
        agg[f"ret_mean_{w}_{prefix}"] = agg[f"ret_1_{prefix}"].rolling(window=w).mean()
        agg[f"ret_std_{w}_{prefix}"] = agg[f"ret_1_{prefix}"].rolling(window=w).std()
        agg[f"volume_mean_{w}_rel_{prefix}"] = agg[f"Volume_mean"].rolling(window=w).mean() / agg[f"Volume_mean"]
        agg[f"volume_std_{w}_rel_{prefix}"] = agg[f"Volume_mean"].rolling(window=w).std() / agg[f"Volume_mean"]
    # Filter rows with date >= 02-01-2008
    agg = agg[agg.index >= pd.to_datetime("02-01-2008", format="%d-%m-%Y")]
    # Drop unwanted columns
    agg = agg.drop(columns=["Close_mean", "Close_std", "Volume_mean", "Volume_std"])
    return agg

# Compute features for each index
folders = {
    "sp500": "sp500",
    "nasdaq": "nasdaq",
    "nyse": "nyse",
    "forbes2000": "forbes2000"
}
features = []
for prefix, folder in folders.items():
    input_folder = os.path.join("data", "cleaned", folder, "csv")
    agg = compute_index_features(input_folder, prefix)
    features.append(agg)

# Merge all features on the date index
from functools import reduce
df_final = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how="outer"), features)
df_final = df_final.sort_index()

# Save to CSV
os.makedirs("data/enriched", exist_ok=True)
df_final.to_csv("data/enriched/market_indexes_aggregated.csv")
print("Aggregated market index features saved to data/enriched/market_indexes_aggregated.csv")

Aggregated market index features saved to data/enriched/market_indexes_aggregated.csv
